In [2]:
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import ssl

class YouTubeDownloader:
    def __init__(self, video_url):
        self.video_url = video_url

    def download_video(self, output_path):
        try:
            # Disable SSL verification
            ssl._create_default_https_context = ssl._create_unverified_context
            yt = YouTube(self.video_url)
            stream = yt.streams.get_highest_resolution()
            stream.download(output_path)
            print(f"Video downloaded to {output_path}")
        except Exception as e:
            print(f"Error downloading video: {e}")

    def download_captions(self, output_path):
        try:
            # Disable SSL verification
            ssl._create_default_https_context = ssl._create_unverified_context
            srt = YouTubeTranscriptApi.get_transcript(self.video_url)
            with open(output_path, "w", encoding="utf-8") as f:
                for caption in srt:
                    f.write("{}\n".format(caption['text']))
            print(f"Captions downloaded to {output_path}")
        except Exception as e:
            print(f"Error downloading captions: {e}")

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=DhmZ6W1UAv4"
   # video_url2 = "ikh3ncJZPTU"
    output_video_path = "Drone_Tracking_1.mp4"
   # output_captions_path = "FiveEyes_captions1.srt"

    downloader = YouTubeDownloader(video_url)
   # downloader2 = YouTubeDownloader(video_url2)
    downloader.download_video(output_video_path)
   # downloader2.download_captions(output_captions_path)


Video downloaded to Drone_Tracking_1.mp4


In [3]:
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import ssl

class YouTubeDownloader:
    def __init__(self, video_url):
        self.video_url = video_url

    def download_video(self, output_path):
        try:
            # Disable SSL verification
            ssl._create_default_https_context = ssl._create_unverified_context
            yt = YouTube(self.video_url)
            stream = yt.streams.get_highest_resolution()
            stream.download(output_path)
            print(f"Video downloaded to {output_path}")
        except Exception as e:
            print(f"Error downloading video: {e}")

    def download_captions(self, output_path):
        try:
            # Disable SSL verification
            ssl._create_default_https_context = ssl._create_unverified_context
            srt = YouTubeTranscriptApi.get_transcript(self.video_url)
            with open(output_path, "w", encoding="utf-8") as f:
                for caption in srt:
                    f.write("{}\n".format(caption['text']))
            print(f"Captions downloaded to {output_path}")
        except Exception as e:
            print(f"Error downloading captions: {e}")

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=YrydHPwRelI"
   # video_url2 = "ikh3ncJZPTU"
    output_video_path = "Drone_Tracking_2.mp4"
   # output_captions_path = "FiveEyes_captions1.srt"

    downloader = YouTubeDownloader(video_url)
   # downloader2 = YouTubeDownloader(video_url2)
    downloader.download_video(output_video_path)
   # downloader2.download_captions(output_captions_path)


Video downloaded to Drone_Tracking_2.mp4


In [4]:
pip install opencv-python numpy tensorflow

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/05/58/7ee92b21cb98689cbe28c69e3cf8ee51f261bfb6bc904ae578736d22d2e7/opencv_python-4.8.1.78-cp37-abi3-macosx_10_16_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 3.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import cv2
import numpy as np
import os

def detect_drones(video_path, output_folder, yolo_weights, yolo_config, yolo_classes):
    net = cv2.dnn.readNet(yolo_weights, yolo_config)
    classes = []
    with open(yolo_classes, "r") as f:
        classes = [line.strip() for line in f.readlines()]

    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        height, width, _ = frame.shape
        frame_width = width
        frame_height = height

        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(net.getUnconnectedOutLayersNames())

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 0:  # Assuming drone class is at index 0
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        output_file = os.path.join(output_folder, 'output.avi')
        out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_directory = "/Users/muhabelgamal/Documents/NJIT/CS370AI/Drone_Tracking_2.mp4/"  # Change this to the directory containing your videos
    output_directory = "Detections"  # Change this to the desired output directory
    yolo_weights_file = "/Users/muhabelgamal/Documents/NJIT/CS370AI/yolov3.weights"  # Change this to the path of YOLO weights file
    yolo_config_file = "/Users/muhabelgamal/Documents/NJIT/CS370AI/yolov3.cfg"  # Change this to the path of YOLO configuration file
    yolo_classes_file = "/Users/muhabelgamal/Documents/NJIT/CS370AI/coco.names"  # Change this to the path of YOLO classes file

    videos = [f for f in os.listdir(video_directory) if f.endswith(('.mp4', '.avi'))]

    for video in videos:
        video_path = os.path.join(video_directory, video)
        output_folder = os.path.join(output_directory, video.split('.')[0])
        os.makedirs(output_folder, exist_ok=True)

        detect_drones(video_path, output_folder, yolo_weights_file, yolo_config_file, yolo_classes_file)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/muhabelgamal/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/c8/r5_15f0d253fhwl3qpwdr7rm0000gn/T/ipykernel_25981/1921291.py", line 80, in <module>
    detect_drones(video_path, output_folder, yolo_weights_file, yolo_config_file, yolo_classes_file)
  File "/var/folders/c8/r5_15f0d253fhwl3qpwdr7rm0000gn/T/ipykernel_25981/1921291.py", line -1, in detect_drones
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/muhabelgamal/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/muhabelgamal/Library/Python/3.11/lib/python/site-packages/IPython/core/ultratb.py", 

In [22]:
pip install protos


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
import cv2
import os
from imageai.Detection import VideoObjectDetection

# Load pre-trained YOLO model
yolo_detector = VideoObjectDetection()
yolo_detector.setModelTypeAsYOLOv3()
yolo_detector.setModelPath("/Users/muhabelgamal/Documents/NJIT/CS370AI/yolov3.weights")
yolo_detector.loadModel()

# Function to detect drones in a frame
def detect_drones(frame):
    # Perform object detection
    detections = yolo_detector.detectObjectsFromImage(input_image=frame, input_type="array")

    # Filter detections for drones
    drone_detections = [d["name"] for d in detections if d["name"] == "drone"]
    return drone_detections

# Function to process a video and save frames with detections
def process_video(video_path):
    video_capture = cv2.VideoCapture(video_path)
    frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create a directory to store detections
    os.makedirs("detections", exist_ok=True)

    for i in range(frame_count):
        ret, frame = video_capture.read()
        if not ret:
            break

        # Detect drones in the frame
        detections = detect_drones(frame)

        # If there are detections, save the frame
        if len(detections) > 0:
            cv2.imwrite(f"detections/frame_{i}.png", frame)

    video_capture.release()

# Process multiple videos in a directory
def process_videos_in_directory(directory_path):
    video_files = [f for f in os.listdir(directory_path) if f.endswith(".mp4")]

    for video_file in video_files:
        video_path = os.path.join(directory_path, video_file)
        process_video(video_path)

# Example usage
videos_directory = "/Users/muhabelgamal/Documents/NJIT/CS370AI/Drone_Tracking_1.mp4/Drone Tracking 1.mp4"
process_videos_in_directory(videos_directory)


ValueError: Invalid model file yolov3.weights. Please parse in a '.pt' and '.pth' model file.

In [36]:
pip install imageai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 1.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [44]:
import cv2
import numpy as np

# Load YOLO model
net = cv2.dnn.readNet("/Users/muhabelgamal/Documents/NJIT/CS370AI/yolov3.weights", "/Users/muhabelgamal/Documents/NJIT/CS370AI/yolov3.cfg")
classes = []
with open("/Users/muhabelgamal/Documents/NJIT/CS370AI/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getUnconnectedOutLayersNames()

# Function to detect drones in a frame
# Function to detect drones in a frame
def detect_drones(frame):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "drone":
                center_x, center_y, w, h = (detection[:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Handle the case where indices is a tuple
    indices = indices[0] if len(indices) > 0 else []

    drone_detections = [boxes[i] for i in indices] if indices else []
    return drone_detections


# Function to process a video and save frames with detections
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create a directory to store detections
    os.makedirs("detections", exist_ok=True)

    for i in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        # Detect drones in the frame
        detections = detect_drones(frame)

        # If there are detections, save the frame
        if len(detections) > 0:
            cv2.imwrite(f"detections/frame_{i}.png", frame)

    cap.release()

# Process multiple videos in a directory
def process_videos_in_directory(directory_path):
    video_files = [f for f in os.listdir(directory_path) if f.endswith(".mp4")]

    for video_file in video_files:
        video_path = os.path.join(directory_path, video_file)
        process_video(video_path)

# Example usage
videos_directory = "/Users/muhabelgamal/Documents/NJIT/CS370AI/Drone_Tracking_1.mp4"
process_videos_in_directory(videos_directory)


In [45]:
process_videos_in_directory(videos_directory)
